In [36]:
from aicsimageio import AICSImage

In [37]:
img = AICSImage("/Users/danielt/Downloads/AICS-12_143.ome.tif")

In [38]:
data = img.get_image_data()
data.shape

(1, 9, 60, 624, 924)

In [51]:
import shutil
import zarr
import pathlib
import numpy
import os
from ome_zarr.writer import write_image
from ome_zarr.scale import Scaler
from ome_zarr.io import parse_url

name = "z0.zarr"
mypath = pathlib.Path("./" + name)
shutil.rmtree(mypath)
print(mypath)
store = parse_url(mypath, mode="w").store
print(store)
root = zarr.group(store=store)
print(root)
group = root.create_group("image0")
scaler = Scaler()
ch = []
for i in range(data.shape[1]):
    ch.append({"active": True,
        "coefficient": 1,
        "color": "0000FF",
        "family": "linear",
        "inverted": False,
        "label": f"Channel{i}",
        "window": {
            "end": 1500,
            "max": 65535,
            "min": 0,
            "start": 0
        }
    })

omero = {"id": 1,                              # ID in OMERO
  "name": "image0",                # Name as shown in the UI
  "version": "0.1",                     # Current version
  "channels": ch,
  "rdefs": {
    "defaultT": 0,                    # First timepoint to show the user
    "defaultZ": data.shape[2]//2,                  # First Z section to show the user
    "model": "color"                  # "color" or "greyscale"
  }
}
write_image(data, group, chunks=(256,256), scaler=scaler, omero=omero)
print(os.listdir(mypath))
print(os.listdir(mypath / "image0"))

z0.zarr
<zarr.hierarchy.Group '/'>
['image0', '.zgroup']
['.zattrs', '.zgroup', '0', '1', '4', '3', '2']


In [44]:
from ome_zarr.reader import Multiscales, Reader
reader = Reader(parse_url(mypath / "image0"))
node = list(reader())[0]
print(node.data[0].shape)
# assert node.data[0].chunks == ((1,), (2,), (1,), (128, 128), (128, 128))

(1, 9, 60, 624, 924)


In [45]:
# levels
print(len(node.data))

5


In [46]:
print(node.data[1].shape)
print(node.data[2].shape)
print(node.data[3].shape)
print(node.data[4].shape)
print(node.data[4].dtype)
print(node.data[4][0].shape)


(1, 9, 60, 462, 312)
(1, 9, 60, 156, 231)
(1, 9, 60, 115, 78)
(1, 9, 60, 39, 57)
uint16
(9, 60, 39, 57)


In [35]:
import nbvv
nbvv.volshow(node.data[0][0].compute(), spacing=(1.0, 1.0, 4.0))

VolumeWidget(dimensions={'tile_width': 341, 'tile_height': 204, 'rows': 10, 'cols': 6, 'atlas_width': 2046, 'a…